Set 2: [PCA/Hyperparameter/CV]
==

Task 0
---
Data preparation

In [1]:
import pandas as pd
# training data import
cs_training = pd.read_csv("D:/课程学习资料/研一M3/Machine Learning for Finance/Homeworks/Set 1/archive/cs-training.csv")
# fill-in the missing values using the column means
cs_training = cs_training.fillna(cs_training.mean())
# X_train Y_train
from sklearn.preprocessing import StandardScaler
X_train = cs_training.iloc[:,list(range(2,12))]
Y_train = cs_training.iloc[:,1]

Task 1
--
Extract a few (>2) features using PCA method.

Using the selected features from above, we are going to apply LR / SVM / decision tree (or any other algorithm).

Implement the methods using pipeline. (PML p185)

Use grid search for finding optimal hyperparameters. (PML p199). In the search, apply 5-fold cross-validation.

**To complete the Task, I will combine pipeline and grid search together.**

Task 1.1
---
PCA + LR

In [2]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(),
                        LogisticRegression(random_state=1))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

n_components=[3,4,5,6]

param_grid = [{'logisticregression__penalty': ['l1'], 
               'logisticregression__C': param_range,
               'logisticregression__solver':['liblinear'],
               'pca__n_components':n_components},
              {'logisticregression__penalty': ['l2'], 
               'logisticregression__C': param_range,
               'logisticregression__solver':['lbfgs'],
               'pca__n_components':n_components}]

gs = GridSearchCV(estimator=pipe_lr, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs =-1)

gs = gs.fit(X_train, Y_train)

print(gs.best_score_)
print(gs.best_params_)

0.9333266666666666
{'logisticregression__C': 0.0001, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs', 'pca__n_components': 3}


Task 1.2
---
PCA + SVM

**In fact, it is hard to converge, so I have to reduce the number of hyperparameter and max_inter.**

In [48]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.svm import SVC
pipe_svc = make_pipeline(StandardScaler(),
                         PCA(),
                         SVC(random_state=1, max_iter = 1000)) 

param_range = [0.001, 0.01, 0.1]

n_components=[3,4]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear'],
               'pca__n_components':n_components},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf'],
               'pca__n_components':n_components}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5)

gs = gs.fit(X_train, Y_train)

print(gs.best_score_)
print(gs.best_params_)

0.7599933333333333
{'pca__n_components': 4, 'svc__C': 0.001, 'svc__kernel': 'linear'}


Task 1.3
---
PCA + decision tree

In [53]:
from sklearn.tree import DecisionTreeClassifier

pipe_tree = make_pipeline(StandardScaler(),
                 PCA(),
                 DecisionTreeClassifier(random_state=0,criterion='gini'))
                 
n_components=[3,4,5,6]

param_grid=[{'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, None],
             'pca__n_components':n_components}]

gs = GridSearchCV(estimator=pipe_tree,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5)

gs = gs.fit(X_train, Y_train)

print(gs.best_score_)
print(gs.best_params_)

0.9361
{'decisiontreeclassifier__max_depth': 4, 'pca__n_components': 3}
